In [78]:
# on vous donne la solution toute faite !
from pulp import *
import numpy as np
from gen_random import gen_random as gen_r

In [79]:
from dijkstar import Graph, find_path
import random
import numpy as np



def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra



def input_output_contrainte(matrice, max_value):
    for i,line in enumerate(matrice):
        arr = np.where(line == 0)[0]
        arr = np.delete(arr,np.where(arr == i))
        while len(arr) > len(matrice)-3:
            index = random.choice(arr)
            matrice[i][index] = random.randint(1,max_value)
            arr = np.delete(arr,np.where(arr == index))
    for i in range(len(matrice)):
        for j in range(len(matrice)):
            if matrice[i][j] != 0:
                matrice[j][i] = matrice[i][j]
    return matrice


#Génère une matrice aléatoire de taille n
def gen_graph(size, max_value, prob):
    matrice = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            if random.uniform(0,1)<prob:
                matrice[i][j] = random.randint(1,max_value)
                matrice[j][i] = matrice[i][j]
            if j == i:
                matrice[i][i] = 0
    input_output_contrainte(matrice, max_value)
    return matrice



def random_city(graph, nb_city):
    if (nb_city <= len(graph)):
        return random.sample(range(len(graph)),nb_city)





def cities_complet(graph, cities):
    graph_cities = np.zeros((len(cities),len(cities)))
    graph_dji = convert_graph(graph)
    for index_city, city in enumerate(cities):
        for index_city2, city2 in enumerate(cities):
            graph_cities[index_city][index_city2] = graph[city][city2]
    for index, city in enumerate(graph_cities):
        liaisons = np.where(city == 0)
        liaisons = np.delete(liaisons,np.where(liaisons[0] == index))
        for col in liaisons:
            shortest_path = find_path(graph_dji,cities[index],cities[col])
            graph_cities[index][col] = shortest_path.total_cost
            
    return graph_cities



def gen_random(size_graph,size_cities, max_value, prob):
    """_summary_

    Args:
        size_graph (int): taille du graph de toutes les villes
        size_cities (int): taille des vilels par lesquels il faut passer
        max_value (int): valeur max des poids Xij
        prob (float): probabilité d'une connection entre 2 villes

    Returns:
        graph: _description_
    """
    graph = gen_graph(size_graph,max_value,prob)
    cities = random_city(graph,size_cities)
    cities_c = cities_complet(graph, cities)


    return graph,cities,cities_c




In [80]:
size_graph=10
size_cities=5
max_value=10
prob=0.2

random_graph,cities,graph_complet_cities = gen_random(size_graph,size_cities, max_value, prob)

In [81]:
print(random_graph)
print(cities)
print(graph_complet_cities)

[[ 0. 10.  4.  7.  0.  0.  8.  0.  0.  0.]
 [10.  0.  0.  4.  4.  5.  0.  0.  0.  0.]
 [ 4.  0.  0.  0. 10.  0.  0.  6.  0.  0.]
 [ 7.  4.  0.  0.  9.  0.  0.  0.  0.  3.]
 [ 0.  4. 10.  9.  0.  0.  0.  9.  0.  6.]
 [ 0.  5.  0.  0.  0.  0.  3.  0.  0.  0.]
 [ 8.  0.  0.  0.  0.  3.  0.  3.  8.  4.]
 [ 0.  0.  6.  0.  9.  0.  3.  0.  6.  3.]
 [ 0.  0.  0.  0.  0.  0.  8.  6.  0.  0.]
 [ 0.  0.  0.  3.  6.  0.  4.  3.  0.  0.]]
[9, 0, 2, 4, 1]
[[ 0.  0.  0.  6.  0.]
 [ 0.  0.  4.  0. 10.]
 [ 0.  4.  0. 10.  0.]
 [ 6.  0. 10.  0.  4.]
 [ 0. 10.  0.  4.  0.]]


In [82]:
def contrainte(matrice):
    """
    Contrainte : Double somme de x_ij supérieur ou égale à 2
    """
    Tableau_ligne = []
    for i in range(len(matrice)):
        count_non_nul = 0
        for j in range(len(matrice)):
            if matrice[i][j] != np.inf:
                count_non_nul += 1
        Tableau_ligne.append(count_non_nul)
    return Tableau_ligne

contrainte(random_graph)

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [83]:
#import libraries
%matplotlib inline
import pulp
import pandas as pd
from scipy.spatial import distance_matrix
from matplotlib import pyplot as plt
import time
import copy
print(graph_complet_cities)

#This function takes locations as input and plot a scatter plot
def plot_fig(loc,heading="plot"):
    plt.figure(figsize=(10,10))
    for i,row in loc.iterrows():
        if i==0:
            plt.scatter(row["x"],row["y"],c='r')
            plt.text(row["x"]+0.2, row["y"]+0.2, 'DELHI (depot) ')
        else:
            plt.scatter(row["x"], row["y"], c='black')
            plt.text(row["x"] + 0.2, row["y"] + 0.2,full_data.loc[i]['CITy'] )
        plt.ylim(6,36)
        plt.xlim(66,96)
        plt.title(heading)
# this function find all the subtour in the LP solution.
def get_plan(r0):
    r=copy.copy(r0)
    route = []
    while len(r) != 0:
        plan = [r[0]]
        del (r[0])
        l = 0
        while len(plan) > l:
            l = len(plan)
            for i, j in enumerate(r):
                if plan[-1][1] == j[0]:
                    plan.append(j)
                    del (r[i])
        route.append(plan)
    return(route)

no_of_locs = len(graph_complet_cities)


dis_mat= graph_complet_cities

start_t_1=time.time()
model=pulp.LpProblem('tsp',pulp.LpMinimize)
#define variable
x=pulp.LpVariable.dicts("x",((i,j) for i in range(no_of_locs) \
                                for j in range(no_of_locs)),\
                           cat='Binary')
#set objective
model+=pulp.lpSum(dis_mat[i][j]* x[i,j] for i in range(no_of_locs) \
                      for j in range(no_of_locs))
# st constraints
for i in range(no_of_locs):
    model+=x[i,i]==0
    model+=pulp.lpSum(x[i,j] for j in range(no_of_locs))==1
    model += pulp.lpSum(x[j, i] for j in range(no_of_locs)) == 1
    
status=model.solve()

route=[(i,j) for i in range(no_of_locs) \
           for j in range(no_of_locs) if pulp.value(x[i,j])==1]
route_plan=get_plan(route)
subtour=[]

while len(route_plan)!=1:
    for i in range(len(route_plan)):
            print(route_plan[i])
    model+=pulp.lpSum(x[route_plan[i][j][0],route_plan[i][j][1]]\
                              for j in range(len(route_plan[i])))<=\
                              len(route_plan[i])-1
    status=model.solve()
    route = [(i, j) for i in range(no_of_locs) \
                 for j in range(no_of_locs) if pulp.value(x[i, j]) == 1]
    route_plan = get_plan(route)
    
    subtour.append(len(route_plan))


print("-----------------")
print(status,pulp.LpStatus[status],pulp.value(model.objective))
print(route_plan)
print("no. of times LP model is solved = ",len(subtour))
print("subtour log (no. of subtours in each solution))",subtour)
print("Time taken by DFJ formulation = ", time.time()-start_t_1)

[[ 0.  0.  0.  6.  0.]
 [ 0.  0.  4.  0. 10.]
 [ 0.  4.  0. 10.  0.]
 [ 6.  0. 10.  0.  4.]
 [ 0. 10.  0.  4.  0.]]
[(0, 2), (2, 4), (4, 0)]
[(1, 3), (3, 1)]
-----------------
1 Optimal 4.0
[[(0, 2), (2, 4), (4, 3), (3, 1), (1, 0)]]
no. of times LP model is solved =  1
subtour log (no. of subtours in each solution)) [1]
Time taken by DFJ formulation =  0.06701421737670898
